In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.neural_network import MLPClassifier
import math
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, train_test_split
import seaborn as sns

In [2]:
df = pd.read_csv('./finalcsvdropped.csv')
df = df.drop(df.iloc[:, 0:1], axis = 1)

/var/folders/dk/ntqqxtcj77d6kpjl7pz6dbbr0000gn/T/ipykernel_2845/492526845.py:1: DtypeWarning: Columns (11,14,20,24,25) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./finalcsvdropped.csv')


In [3]:
df.head()

,timestamp_local,WEATHER_DELAY,app_temp,azimuth,clouds,dewpt,dhi,dni,elev_angle,ghi,...,timestamp_utc,ts,uv,vis,weather/description,weather/icon,weather/code,wind_dir,wind_gust_spd,wind_spd
0,2018-01-01T19:15:00,0.0,-14.1,263.30,25.0,-20.7,0.0,0.00,-28.59,0.0,...,2018-01-02T00:15:00,1.514852e+09,0.0,16.0,Scattered clouds,c02n,802.0,270.0,3.9,3.60
1,2018-01-01T00:00:00,0.0,-22.3,0.11,18.0,-20.6,0.0,0.00,-72.36,0.0,...,2018-01-01T05:00:00,1.514783e+09,0.0,16.0,Few clouds,c02n,801.0,325.0,8.0,6.82
2,2018-01-01T23:45:00,9.0,-17.5,349.78,25.0,-16.9,0.0,0.00,-71.26,0.0,...,2018-01-02T04:45:00,1.514868e+09,0.0,16.0,Scattered clouds,c02n,802.0,250.0,5.7,5.09
3,2018-01-01T19:45:00,0.0,-14.1,267.93,25.0,-20.0,0.0,0.00,-34.25,0.0,...,2018-01-02T00:45:00,1.514854e+09,0.0,16.0,Scattered clouds,c02n,802.0,250.0,3.9,3.60
4,2018-01-01T14:45:00,0.0,-14.6,218.90,25.0,-21.0,69.7,616.27,15.50,228.8,...,2018-01-01T19:45:00,1.514836e+09,1.2,16.0,Scattered clouds,c02d,802.0,320.0,11.3,7.20


In [4]:
def predict_mlp_classifier_precip_rate(df):
  df = df.dropna()
  X = df.drop('WEATHER_DELAY', axis=1 )  
  y = df['WEATHER_DELAY']  
  X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.30,random_state=10) 
  model = MLPClassifier(hidden_layer_sizes=(8,9,9), activation='relu', max_iter=5000)
  model.fit(X_train, y_train)
  print("score:", model.score(X_test, y_test)) # this is for comparing delay to precipitation (where delay =0 means no delay and delay = -1 means cancelled)

In [5]:
predict_mlp_classifier_precip_rate(df) #score: 0.46704565642209983

ValueError: could not convert string to float: '2018-12-20T01:45:00'

In [ ]:
def predict_mlp_classifier_temp(df):
  df_train = df.iloc[:5629, :] #x is temp and y is delay
  df_test = df.iloc[5629:, :]
  for i in range(len(df['DEP_DELAY'])):
    if df['DEP_DELAY'][i]==-1:
      df['DEP_DELAY'][i]=300
  df_train_y = df_train[['DEP_DELAY']]
  df_train_x = df_train[['temp']]
  model = MLPClassifier(hidden_layer_sizes=(8,9,9), activation='relu', max_iter=5000)
  model.fit(df_train_x, df_train_y)
  print("score:", model.score(df_train_x, df_train_y)) # this is for comparing delay to temp 

: 

In [ ]:
predict_mlp_classifier_temp(formatted_df) #score: 0.46722330786995914

: 

In [ ]:
def predict_logistic_regression(df):
  features = ['precip_rate', 'wind_spd']
  X = df[features]
  df['cancelled'] = ''
  for i in range(len(df['DEP_DELAY'])): # adding 'cancelled' column
    if(np.isnan(df['DEP_DELAY'][i])):
      df['cancelled'][i]=True
      df['DEP_DELAY'][i] = -1
    #else:
      #df['cancelled'][i]=False
  #y = df['cancelled']
  y = df['DEP_DELAY']
  print(y)
  scaler = StandardScaler()
  X_scaled = scaler.fit_transform(X)
  log = LogisticRegression(class_weight='balanced')
  log.fit(X_scaled,y)
  print("score:", log.score(X_scaled,y))
  return df #score: 0.0014212115828744004

: 

In [ ]:
formatted_df = format_df(df)

: 

In [ ]:
predict_logistic_regression(df)

: 

In [ ]:
def predict_logistic_regression_with_negative_delays_as_0(df):
  features = ['precip_rate', 'wind_spd']
  X = df[features]
  df['cancelled'] = ''
  for i in range(len(df['DEP_DELAY'])): # adding 'cancelled' column
    if(np.isnan(df['DEP_DELAY'][i])):
      df['cancelled'][i]=True
      df['DEP_DELAY'][i] = -1
    if(df["DEP_DELAY"][i] < 0):
      df["DEP_DELAY"][i] = 0
    #else:
      #df['cancelled'][i]=False
  #y = df['cancelled']
  y = df['DEP_DELAY']
  print(y)
  scaler = StandardScaler()
  X_scaled = scaler.fit_transform(X)
  log = LogisticRegression(class_weight='balanced', max_iter=10000)
  log.fit(X_scaled,y)
  print("score:", log.score(X_scaled,y))
  return df #score: 0.0013323858589447503

: 

In [ ]:
formatted_df = format_df(df)

: 

In [ ]:
predict_logistic_regression_with_negative_delays_as_0(formatted_df)

: 

In [ ]:
def predict_logistic_regression_without_cancellations(df):
  features = ['precip_rate', 'wind_spd']
  X = df[features]
  df.dropna()
  for i in range(len(df['DEP_DELAY'])): 
    if(np.isnan(df['DEP_DELAY'][i])):
      df['DEP_DELAY'][i] = -1
    if(df["DEP_DELAY"][i] < 0):
      df["DEP_DELAY"][i] = 0
  y = df['DEP_DELAY']
  print(y)
  scaler = StandardScaler()
  X_scaled = scaler.fit_transform(X)
  log = LogisticRegression(class_weight='balanced', max_iter=10000)
  log.fit(X_scaled,y)
  print("score:", log.score(X_scaled,y))
  return df #score: 0.0013323858589447503

: 

In [ ]:
formatted_df = format_df(df)

: 

In [ ]:
predict_logistic_regression_without_cancellations(formatted_df)

: 

In [ ]:
def predict_logistic_regression_without_cancellations_no_standardscaler(df):
  features = ['precip_rate', 'wind_spd']
  X = df[features]
  df.dropna()
  for i in range(len(df['DEP_DELAY'])): 
    if(np.isnan(df['DEP_DELAY'][i])):
      df['DEP_DELAY'][i] = -1
    if(df["DEP_DELAY"][i] < 0):
      df["DEP_DELAY"][i] = 0
  y = df['DEP_DELAY']
  print(y)
  #scaler = StandardScaler()
  #X_scaled = scaler.fit_transform(X)
  log = LogisticRegression(class_weight='balanced', max_iter=10000)
  log.fit(X,y)
  print("score:", log.score(X,y))
  return df #score: 0

: 

In [ ]:
predict_logistic_regression_without_cancellations_no_standardscaler(formatted_df)

: 

In [ ]:
df_final = pd.read_csv('/content/drive/My Drive/gateway_final_project/finalcsv.csv')

: 

In [ ]:
df_final.head()

: 

In [ ]:
df_final.shape

: 

In [ ]:
print(df_final['weather/description'])

: 

In [ ]:
df_final['weather/description'].value_counts()

: 

In [ ]:
sns.histplot(data=df_final['weather/description'].value_counts())

: 

In [ ]:
df_final['weather/description'].value_counts().plot(kind='barh')

: 

In [ ]:
print(df_final.loc[df_final['weather/description']=="Overcast clouds"])

: 

In [ ]:
print(df_final.groupby(['weather/description']).mean()['WEATHER_DELAY'])

: 

In [ ]:
df_avg_delay = df_final.groupby(['weather/description']).mean()['WEATHER_DELAY']

: 

In [ ]:
sns.histplot(data=df_avg_delay, x=df_avg_delay[0:,], y=df_avg_delay[0:,])

: 

In [ ]:
df_avg_delay.plot(kind = "bar")

: 

In [ ]:
df_avg_delay_greater_than_15 = df_final.loc[df_final['WEATHER_DELAY'] >= 15].groupby(['weather/description']).mean()['WEATHER_DELAY']

: 

In [ ]:
df_avg_delay_greater_than_15.plot(kind = "bar")

: 

In [ ]:
df_avg_delay_greater_than_60 = df_final.loc[df_final['WEATHER_DELAY'] >= 60].groupby(['weather/description']).mean()['WEATHER_DELAY']

: 

In [ ]:
df_avg_delay_greater_than_60.plot(kind = "bar")

: 

In [ ]:
df_incl_0 = pd.read_csv('/content/drive/My Drive/gateway_final_project/finalcsvdropped (1).csv') # this is weather_delay >= 0

: 

In [ ]:
print(df_incl_0.groupby(['weather/description']).mean()['WEATHER_DELAY'])

: 

In [ ]:
df_avg_delay_0

: 